In [ ]:
pip install xlwt

In [ ]:
pip install xlrd

In [ ]:
!pip install opencv-contrib-python==4.5.1.48

In [ ]:
!pip install facenet-pytorch



In [ ]:
pip install xlutils

In [ ]:
#pip install mtcnn

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()



In [ ]:
import datetime
import xlwt
import xlrd
from xlwt import Workbook 
from matplotlib import pyplot
#from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from keras.models import load_model
from matplotlib.patches import Circle
import cv2
import numpy as np
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine
import os
from xlutils.copy import copy
from facenet_pytorch import MTCNN
import torch

In [ ]:

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

In [ ]:
import tensorflow as tf
tf.compat.v1.debugging.set_log_device_placement(True)
print(tf.config.list_physical_devices('GPU'))


In [ ]:
tf.test.gpu_device_name()

** TPU **

In [ ]:
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#with tpu_strategy.scope():
encoder_model = '../input/facenet-keras/facenet_keras.h5'

detector=MTCNN(keep_all=False, device='cuda:0')
#detector=MTCNN()
face_encoder = load_model(encoder_model)

people_dir = '../input/photos'
encoding_dict = dict()

"""### Encoding of face using FaceNet"""

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

"""### Extract Face for encoding"""

def get_face(img, box):
    [[x1, y1, width, height]] = box
    x1, y1 ,x2,y2= int(x1), int(y1),int(width),int(height)
    #x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)
"""def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)"""


def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

l2_normalizer = Normalizer('l2')

### collect daywise attendance by checking through a list of ppl
def export(a):

    ct = datetime.datetime.now()
    ws = xlrd.open_workbook('../input/nnnnnnn/names_list.xls',formatting_info=True)
    sheet = ws.sheet_by_index(0)
    wb=copy(ws)
    names_sheet=wb.get_sheet(0)
    row_s=sheet.nrows
    col_s=sheet.ncols
    names_sheet.write(0,col_s,str(ct.year)+"_"+str(ct.month)+"_"+str(ct.day)+"_"+str(ct.hour))
    for i in range(1,row_s):

        if sheet.cell_value(i,0) in a:
            names_sheet.write(i, col_s,"P")

        else:  
            names_sheet.write(i, col_s,"A")


    wb.save("../input/nnnnnnn/names_list.xls") 





In [ ]:
#free_gpu_cache()
with tpu_strategy.scope():
    for person_name in os.listdir(people_dir):
        person_dir = os.path.join(people_dir, person_name)
        encodes = []
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = cv2.imread(img_path)

            boxes,probs = detector.detect(img)
            #print(boxes)
            results = detector.detect_faces(img)
            if len(boxes)>0:

                #res = max(results, key=lambda b: b['box'][2] * b['box'][3])
                #print(res)
                face, _, _ = get_face(img, boxes)
                #plt.imshow(face)
                face = normalize(face)
                face = cv2.resize(face,(160,160))
                encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
                encodes.append(encode)
        if encodes:
            encode = np.sum(encodes, axis=0)
            encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
            encoding_dict[person_name] = encode
        #free_gpu_cache()

    recognition_t=0.6
    confidence_t=0.99

In [ ]:

#free_gpu_cache()
video =cv2.VideoCapture('../input/videoo/WIN_20210324_18_59_11_Pro.mp4')
present_candidates=[]

#sr = cv2.dnn_superres.DnnSuperResImpl_create()

#path = "../input/nnnnnnn/LapSRN_x8.pb"
#sr.readModel(path)
#sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
# Set the desired model and scale to get correct pre- and post-processing
#sr.setModel("lapsrn", 8)

check=True


if check:
    check,frame=video.read()
    
    frame=sr.upsample(frame)
    



    faces=detector.detect_faces(frame)
    print("chh")
    
    if faces !=[]:
        for person in faces:
            bounding_box=person["box"]
            keypoints=person["keypoints"]
#             if person['confidence'] < confidence_t:
#                 continue
            face, pt_1, pt_2 = get_face(frame, person['box'])
            encode = get_encode(face_encoder, face,(160,160))
            encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
            name = 'unknown'


            distance = float("inf")


            for (db_name, db_enc) in encoding_dict.items():

                dist = cosine(db_enc, encode)

                if dist < recognition_t and dist < distance:

                    name = db_name
                    distance = dist
                    if name not in present_candidates:
                        present_candidates.append(name)


            if name == 'unknown':
                cv2.rectangle(frame, pt_1, pt_2, (0, 0, 255), 2)
                cv2.putText(frame,name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

            else:
                cv2.rectangle(frame, pt_1, pt_2, (0, 255, 0), 2)
                cv2.putText(frame,name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)

            #free_gpu_cache()
    print(present_candidates)
    




    
    #cv2.waitkey(0)

        #if cv2.waitKey(5) & 0xFF == ord('q'):
              #break



#video.release()
#cv2.destroyAllWindows()


